In [5]:
import pandas as pd
import numpy as np
import torch
from chronos import BaseChronosPipeline
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

In [6]:


class FinancialTimeSeriesPredictor:
    def __init__(self, model_name="amazon/chronos-bolt-base", device="cuda"):
        """Initialize the financial predictor with Chronos-Bolt model"""
        self.model_name = model_name
        self.device = device if torch.cuda.is_available() else "cpu"
        self.pipeline = None
        self.df = None
        self.features_df = None
        
    def load_data(self, csv_file_path, date_col="DATE", close_col="CLOSE PRICE"):
        """Load and preprocess financial data"""
        print("Loading financial data...")
        
        # Load CSV
        self.df = pd.read_csv(csv_file_path)
        self.df.columns = self.df.columns.str.strip()
        
        # Convert date and sort
        self.df[date_col.strip()] = pd.to_datetime(self.df[date_col.strip()], format='%d-%b-%Y')
        self.df = self.df.sort_values(date_col.strip()).reset_index(drop=True)
        
        # Convert price columns to numeric
        price_cols = ['OPEN PRICE', 'HIGH PRICE', 'LOW PRICE', 'CLOSE PRICE', 'SETTLE PRICE']
        for col in price_cols:
            if col in self.df.columns:
                self.df[col] = pd.to_numeric(self.df[col], errors='coerce')
        
        # Handle volume (remove commas)
        if 'Volume' in self.df.columns:
            self.df['Volume'] = self.df['Volume'].astype(str).str.replace(',', '')
            self.df['Volume'] = pd.to_numeric(self.df['Volume'], errors='coerce')
        
        # Remove rows with missing OHLC data
        self.df = self.df.dropna(subset=price_cols[:4])
        
        print(f"Data loaded: {len(self.df)} records from {self.df[date_col.strip()].min()} to {self.df[date_col.strip()].max()}")
        return self.df
    
    def engineer_features(self, target_col="CLOSE PRICE"):
        """Create comprehensive financial features for better forecasting"""
        print("Engineering financial features...")
        
        df = self.df.copy()
        
        # Price-based features
        df['price_change'] = df[target_col].pct_change()
        df['price_volatility'] = df['price_change'].rolling(window=5).std()
        df['price_momentum'] = df[target_col].rolling(window=3).mean() / df[target_col].rolling(window=10).mean() - 1
        
        # Technical Indicators
        # 1. Moving Averages
        for window in [5, 10, 20]:
            df[f'SMA_{window}'] = df[target_col].rolling(window=window).mean()
            df[f'EMA_{window}'] = df[target_col].ewm(span=window).mean()
        
        # 2. RSI (Relative Strength Index)
        def calculate_rsi(prices, window=14):
            delta = prices.diff()
            gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
            loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
            rs = gain / loss
            return 100 - (100 / (1 + rs))
        
        df['RSI'] = calculate_rsi(df[target_col])
        df['RSI_normalized'] = (df['RSI'] - 50) / 50  # Normalize RSI to [-1, 1]
        
        # 3. MACD
        exp1 = df[target_col].ewm(span=12).mean()
        exp2 = df[target_col].ewm(span=26).mean()
        df['MACD'] = exp1 - exp2
        df['MACD_signal'] = df['MACD'].ewm(span=9).mean()
        df['MACD_histogram'] = df['MACD'] - df['MACD_signal']
        
        # 4. Bollinger Bands
        df['BB_middle'] = df[target_col].rolling(window=20).mean()
        bb_std = df[target_col].rolling(window=20).std()
        df['BB_upper'] = df['BB_middle'] + (2 * bb_std)
        df['BB_lower'] = df['BB_middle'] - (2 * bb_std)
        df['BB_position'] = (df[target_col] - df['BB_lower']) / (df['BB_upper'] - df['BB_lower'])
        
        # 5. Volume-based indicators (if volume available)
        if 'Volume' in df.columns:
            df['Volume_SMA'] = df['Volume'].rolling(window=20).mean()
            df['Volume_ratio'] = df['Volume'] / df['Volume_SMA']
        
        # 6. ATR (Average True Range)
        df['ATR'] = (df['HIGH PRICE'] - df['LOW PRICE']).rolling(window=14).mean()
        df['ATR_normalized'] = df['ATR'] / df[target_col]
        
        # Create feature matrix for forecasting
        feature_columns = [
            target_col, 'SMA_5', 'SMA_10', 'SMA_20', 'EMA_5', 'EMA_10', 
            'RSI_normalized', 'MACD', 'MACD_signal', 'BB_position', 
            'ATR_normalized', 'price_momentum'
        ]
        
        # Add volume features if available
        if 'Volume' in df.columns:
            feature_columns.extend(['Volume_ratio'])
        
        # Select available columns
        available_features = [col for col in feature_columns if col in df.columns]
        self.features_df = df[['DATE'] + available_features].copy()
        
        # Fill missing values
        self.features_df = self.features_df.fillna(method='ffill').fillna(method='bfill')
        
        print(f"Created {len(available_features)} features for forecasting")
        return self.features_df
    
    def load_model(self):
        """Load the Chronos-Bolt model"""
        print(f"Loading {self.model_name} model...")
        
        self.pipeline = BaseChronosPipeline.from_pretrained(
            self.model_name,
            device_map=self.device,
            torch_dtype=torch.bfloat16,
        )
        print("Model loaded successfully!")
        
    def predict(self, prediction_length=30, lookback_window=60):
        """Generate forecasts using Chronos-Bolt (Fixed for Bolt models)"""
        if self.pipeline is None:
            self.load_model()
        
        print(f"Generating forecasts for {prediction_length} periods...")
        
        # Create context - use only close price (Chronos works best with univariate data)
        target_col = 'CLOSE PRICE'
        close_prices = self.features_df[target_col].values
        
        # Take last lookback_window observations
        context_length = min(lookback_window, len(close_prices))
        context = torch.tensor(close_prices[-context_length:], dtype=torch.float32)
        
        # Generate forecasts - Chronos-Bolt uses direct quantile prediction
        try:
            # For Chronos-Bolt models, use predict method without num_samples
            forecast = self.pipeline.predict(
                context=context,
                prediction_length=prediction_length
            )
            
            # Chronos-Bolt returns quantiles directly: [batch_size, prediction_length, num_quantiles]
            forecast_np = forecast[0].cpu().numpy()  # Remove batch dimension
            
            # Extract different quantiles (Chronos-Bolt typically returns multiple quantiles)
            if len(forecast_np.shape) == 2:  # [prediction_length, num_quantiles]
                # Common quantiles: [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
                num_quantiles = forecast_np.shape[1]
                if num_quantiles >= 9:
                    quantiles = np.array([
                        forecast_np[:, 0],  # 0.1 quantile (10th percentile)
                        forecast_np[:, 2],  # 0.3 quantile (30th percentile) 
                        forecast_np[:, 4],  # 0.5 quantile (median)
                        forecast_np[:, 6],  # 0.7 quantile (70th percentile)
                        forecast_np[:, 8]   # 0.9 quantile (90th percentile)
                    ])
                else:
                    # Fallback if fewer quantiles available
                    quantiles = np.array([
                        forecast_np[:, 0],
                        forecast_np[:, num_quantiles//4],
                        forecast_np[:, num_quantiles//2],
                        forecast_np[:, 3*num_quantiles//4],
                        forecast_np[:, -1]
                    ])
                
                mean_forecast = forecast_np[:, num_quantiles//2]  # Use median as mean
            else:
                # If single forecast, create artificial quantiles around it
                mean_forecast = forecast_np
                std_forecast = np.std(forecast_np) if len(forecast_np) > 1 else np.abs(forecast_np * 0.05)
                quantiles = np.array([
                    mean_forecast - 1.5 * std_forecast,
                    mean_forecast - 0.5 * std_forecast,
                    mean_forecast,
                    mean_forecast + 0.5 * std_forecast,
                    mean_forecast + 1.5 * std_forecast
                ])
            
        except Exception as e:
            print(f"Direct predict failed: {e}")
            # Fallback: try predict_quantiles method
            try:
                quantiles_out, mean_out = self.pipeline.predict_quantiles(
                    context=context,
                    prediction_length=prediction_length,
                    quantile_levels=[0.1, 0.3, 0.5, 0.7, 0.9]
                )
                quantiles = quantiles_out[0].cpu().numpy().T  # Transpose to [num_quantiles, prediction_length]
                mean_forecast = mean_out[0].cpu().numpy()
            except Exception as e2:
                print(f"Both methods failed: {e2}")
                raise e2
        
        return {
            'quantiles': quantiles,
            'mean': mean_forecast,
            'context': context.numpy()
        }
    
    def plot_forecast_results(self, forecast_results, prediction_length=30, 
                            title="USD-INR Futures Forecast", save_html=True):
        """Plot comprehensive forecast results with technical indicators"""
        
        # Prepare data for plotting
        historical_dates = self.features_df['DATE'].values
        historical_prices = self.features_df['CLOSE PRICE'].values
        
        # Create future dates
        last_date = pd.to_datetime(historical_dates[-1])
        future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), 
                                   periods=prediction_length, freq='D')
        
        # Get forecast data
        quantiles = forecast_results['quantiles']
        mean_forecast = forecast_results['mean']
        
        # Create subplots
        fig = make_subplots(
            rows=4, cols=1,
            shared_xaxes=True,
            vertical_spacing=0.03,
            subplot_titles=[
                f'{title} - Price Forecast',
                'RSI Indicator',
                'MACD',
                'Volume (if available)'
            ],
            row_heights=[0.5, 0.2, 0.2, 0.1]
        )
        
        # Main price chart with forecast
        lookback = min(60, len(historical_dates))
        recent_dates = historical_dates[-lookback:]
        recent_prices = historical_prices[-lookback:]
        
        # Historical prices
        fig.add_trace(
            go.Scatter(
                x=recent_dates,
                y=recent_prices,
                mode='lines',
                name='Historical Price',
                line=dict(color='blue', width=2)
            ),
            row=1, col=1
        )
        
        # Forecast mean
        fig.add_trace(
            go.Scatter(
                x=future_dates,
                y=mean_forecast,
                mode='lines',
                name='Forecast (Mean)',
                line=dict(color='red', width=2, dash='dash')
            ),
            row=1, col=1
        )
        
        # Prediction intervals
        fig.add_trace(
            go.Scatter(
                x=future_dates,
                y=quantiles[4],  # 90th percentile
                mode='lines',
                name='90% Upper',
                line=dict(color='red', width=1),
                showlegend=False
            ),
            row=1, col=1
        )
        
        fig.add_trace(
            go.Scatter(
                x=future_dates,
                y=quantiles[0],  # 10th percentile
                mode='lines',
                name='90% Lower',
                line=dict(color='red', width=1),
                fill='tonexty',
                fillcolor='rgba(255,0,0,0.2)',
                showlegend=False
            ),
            row=1, col=1
        )
        
        # 50% prediction interval
        fig.add_trace(
            go.Scatter(
                x=future_dates,
                y=quantiles[3],  # 70th percentile
                mode='lines',
                name='50% Upper',
                line=dict(color='orange', width=1),
                showlegend=False
            ),
            row=1, col=1
        )
        
        fig.add_trace(
            go.Scatter(
                x=future_dates,
                y=quantiles[1],  # 30th percentile
                mode='lines',
                name='50% Lower',
                line=dict(color='orange', width=1),
                fill='tonexty',
                fillcolor='rgba(255,165,0,0.3)',
                showlegend=False
            ),
            row=1, col=1
        )
        
        # Add technical indicators
        # RSI
        if 'RSI' in self.features_df.columns:
            fig.add_trace(
                go.Scatter(
                    x=recent_dates,
                    y=self.features_df['RSI'].values[-lookback:],
                    mode='lines',
                    name='RSI',
                    line=dict(color='purple'),
                    showlegend=False
                ),
                row=2, col=1
            )
            
            # RSI levels
            fig.add_hline(y=70, line_dash="dash", line_color="red", row=2, col=1)
            fig.add_hline(y=30, line_dash="dash", line_color="green", row=2, col=1)
        
        # MACD
        if 'MACD' in self.features_df.columns:
            fig.add_trace(
                go.Scatter(
                    x=recent_dates,
                    y=self.features_df['MACD'].values[-lookback:],
                    mode='lines',
                    name='MACD',
                    line=dict(color='blue'),
                    showlegend=False
                ),
                row=3, col=1
            )
            
            if 'MACD_signal' in self.features_df.columns:
                fig.add_trace(
                    go.Scatter(
                        x=recent_dates,
                        y=self.features_df['MACD_signal'].values[-lookback:],
                        mode='lines',
                        name='MACD Signal',
                        line=dict(color='red'),
                        showlegend=False
                    ),
                    row=3, col=1
                )
        
        # Volume (if available)
        if 'Volume' in self.features_df.columns:
            fig.add_trace(
                go.Bar(
                    x=recent_dates,
                    y=self.features_df['Volume'].values[-lookback:],
                    name='Volume',
                    marker_color='lightblue',
                    showlegend=False
                ),
                row=4, col=1
            )
        
        # Update layout
        fig.update_layout(
            title=f'{title} - Chronos-Bolt Forecast',
            xaxis_title='Date',
            yaxis_title='Price',
            height=800,
            showlegend=True,
            legend=dict(
                orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1
            )
        )
        
        if save_html:
            filename = f"{title.lower().replace(' ', '_')}_chronos_forecast.html"
            fig.write_html(filename)
            print(f"Forecast chart saved to {filename}")
        
        fig.show()
        
        return fig




In [7]:
def create_trading_signals(predictor, results):
    """Generate trading signals based on forecast"""
    
    mean_forecast = results['mean']
    current_price = predictor.features_df['CLOSE PRICE'].iloc[-1]
    
    # Simple trading signals
    if len(mean_forecast) >= 7:
        short_term_change = (mean_forecast[6] / current_price - 1) * 100  # 7-day
    else:
        short_term_change = (mean_forecast[min(2, len(mean_forecast)-1)] / current_price - 1) * 100
    
    long_term_change = (mean_forecast[-1] / current_price - 1) * 100  # Full period
    
    print("\n" + "="*40)
    print("TRADING SIGNALS")
    print("="*40)
    
    if short_term_change > 2:
        print("Short-term Signal: STRONG BUY 📈")
    elif short_term_change > 0.5:
        print("Short-term Signal: BUY 📊")
    elif short_term_change < -2:
        print("Short-term Signal: STRONG SELL 📉")
    elif short_term_change < -0.5:
        print("Short-term Signal: SELL 📊")
    else:
        print("Short-term Signal: HOLD ➖")
    
    print(f"Short-term Expected Change: {short_term_change:.2f}%")
    print(f"Full Period Expected Change: {long_term_change:.2f}%")
    
    # Risk analysis
    volatility = np.std(results['quantiles']) / current_price * 100
    print(f"Forecast Volatility: {volatility:.2f}%")
    
    return {
        'short_term_change': short_term_change,
        'long_term_change': long_term_change,
        'volatility': volatility
    }


In [8]:

# Main execution function
def run_financial_forecast(csv_file="Quote-CD-USDINR-15-09-2024-to-15-09-2025.csv", 
                          prediction_days=30):
    """Complete pipeline for financial forecasting"""
    
    # Initialize predictor
    predictor = FinancialTimeSeriesPredictor(
        model_name="amazon/chronos-bolt-base",
        device="cuda"  # Change to "cpu" if no GPU available
    )
    
    # Load and process data
    predictor.load_data(csv_file)
    predictor.engineer_features()
    
    # Generate forecasts
    forecast_results = predictor.predict(
        prediction_length=prediction_days,
        lookback_window=60  # Reduced for better performance
    )
    
    # Plot results
    fig = predictor.plot_forecast_results(
        forecast_results,
        prediction_length=prediction_days,
        title="USD-INR Futures Forecast with Chronos-Bolt"
    )
    
    # Print forecast summary
    mean_forecast = forecast_results['mean']
    current_price = predictor.features_df['CLOSE PRICE'].iloc[-1]
    
    print("\n" + "="*50)
    print("FORECAST SUMMARY")
    print("="*50)
    print(f"Current Price: ₹{current_price:.4f}")
    print(f"Forecast Period: {prediction_days} days")
    print(f"Predicted Price ({prediction_days} days): ₹{mean_forecast[-1]:.4f}")
    print(f"Expected Change: {((mean_forecast[-1] / current_price) - 1) * 100:.2f}%")
    print(f"Price Range (90% confidence): ₹{forecast_results['quantiles'][0][-1]:.4f} - ₹{forecast_results['quantiles'][4][-1]:.4f}")
    
    # Generate trading signals
    signals = create_trading_signals(predictor, forecast_results)
    
    return predictor, forecast_results



In [9]:
# Run the complete pipeline
predictor, results = run_financial_forecast(
    csv_file="Data/Quote-CD-USDINR-15-09-2024-to-15-09-2025.csv",
    prediction_days=5
)

Loading financial data...
Data loaded: 92 records from 2024-09-30 00:00:00 to 2025-09-12 00:00:00
Engineering financial features...
Created 13 features for forecasting
Loading amazon/chronos-bolt-base model...
Model loaded successfully!
Generating forecasts for 5 periods...


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Forecast chart saved to usd-inr_futures_forecast_with_chronos-bolt_chronos_forecast.html



FORECAST SUMMARY
Current Price: ₹88.3200
Forecast Period: 5 days
Predicted Price (5 days): ₹88.9112
Expected Change: 0.67%
Price Range (90% confidence): ₹88.6416 - ₹88.9041

TRADING SIGNALS
Short-term Signal: HOLD ➖
Short-term Expected Change: 0.22%
Full Period Expected Change: 0.67%
Forecast Volatility: 0.48%
